<a href="https://colab.research.google.com/github/micah-shull/LangChain/blob/main/LC_000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**LangChain**
LangChain is a powerful framework designed to simplify building applications that use **large language models (LLMs)**. It's especially useful for tasks like:

* Question answering over documents
* Chatbots
* Code analysis
* Agent-based workflows
* Integration with tools like search engines, APIs, and databases

---

### 🔧 What You Need to Know First

Before diving into LangChain, it's helpful to have a basic understanding of:

* **Python programming**
* How **LLMs** like OpenAI's GPT work at a high level
* Familiarity with APIs and JSON
* Optional: Experience with libraries like `pydantic`, `FastAPI`, or `Transformers`

---

### 🧱 LangChain Core Concepts

Here are the main building blocks of LangChain:

| Concept              | Description                                              |
| -------------------- | -------------------------------------------------------- |
| **LLMs**             | Wraps models like OpenAI, Cohere, etc.                   |
| **Prompts**          | Templates that define the input to the LLM               |
| **Chains**           | Sequences of calls (e.g., prompt → model → output)       |
| **Agents**           | Decision-making systems that use tools to complete tasks |
| **Tools**            | Functions the agent can call (e.g., search, calculator)  |
| **Memory**           | Keeps track of the conversation history                  |
| **Document Loaders** | Load documents from PDFs, web pages, etc.                |
| **Vector Stores**    | Store document embeddings for semantic search            |
| **Retrievers**       | Find relevant documents based on a query                 |

---

### 🧪 How to Start Learning

Here’s a learning plan we can follow. Let me know your pace or focus area:

#### ✅ Step 1: Installation & Setup

* Install: `pip install langchain openai`
* Get OpenAI API key (or use other model providers)

#### ✅ Step 2: Your First Chain

* Build a simple prompt → LLM → response chain

#### ✅ Step 3: Use Documents

* Load documents (e.g., PDF, text)
* Embed them using `FAISS` or `Chroma`
* Create a retriever and ask questions

#### ✅ Step 4: Build an Agent

* Give it tools (search, math, retrieval)
* Use ReAct pattern for reasoning

#### ✅ Step 5: Add Memory

* Add chat memory to track conversations

#### ✅ Step 6: Build a Full App

* Combine chains, agents, and retrievers into a useful app
* Possibly deploy with Streamlit or FastAPI



## Environment Information

In [15]:
# !pip install --upgrade \
#     langchain \
#     langchain-community \
#     langchain-core \
#     openai \
#     chromadb \
#     faiss-cpu \
#     tiktoken

# !pip install langchain-openai

In [1]:
import importlib
import pkg_resources

# List of relevant packages
packages = [
    "langchain",
    "openai",
    "chromadb",
    "faiss-cpu",
    "faiss-gpu",
    "tiktoken",
    "pydantic",
    "numpy",
    "pandas",
    "requests",
    "httpx",
    "tenacity"
]

def check_versions(packages):
    for pkg in packages:
        try:
            version = pkg_resources.get_distribution(pkg).version
            print(f"{pkg}: {version}")
        except pkg_resources.DistributionNotFound:
            print(f"{pkg}: NOT INSTALLED")

check_versions(packages)

langchain: 0.3.25
openai: 1.82.1
chromadb: 1.0.12
faiss-cpu: 1.11.0
faiss-gpu: NOT INSTALLED
tiktoken: 0.9.0
pydantic: 2.11.4
numpy: 2.0.2
pandas: 2.2.2
requests: 2.32.3
httpx: 0.28.1
tenacity: 9.1.2


In [6]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv("/content/API_KEYS.env")

# Now it's available as an environment variable
print("API Key loaded:", os.getenv("OPENAI_API_KEY")[:8] + "...")

API Key loaded: sk-proj-...


In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# Set up the LLM (you'll be prompted for your API key if not set)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# Build a prompt template
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")

# Chain it together
chain = prompt | llm | StrOutputParser()

# Run it
response = chain.invoke({"topic": "AI and bananas"})
print(response)

Why did the AI break up with the banana? Because it couldn't handle its a-peeling personality!



## 🧩 Step 1: `ChatPromptTemplate` — What Is It and Why Use It?

---

### 🔍 **What Is a Prompt Template?**

A **prompt template** in LangChain is a **reusable blueprint** for how you structure your input to the LLM. Instead of hardcoding a full prompt like:

```python
"Tell me a joke about AI and bananas"
```

You create a **template with variables**, like:

```python
"Tell me a joke about {topic}"
```

This lets you **plug in different values** (`"AI and bananas"`, `"cats"`, `"quantum physics"`, etc.) at runtime.

---

### 💬 `ChatPromptTemplate` vs `PromptTemplate`

There are two main types:

* `PromptTemplate`: for **text-completion** models (like `text-davinci-003`)
* `ChatPromptTemplate`: for **chat models** like `gpt-3.5-turbo`, `gpt-4`

Since OpenAI chat models expect structured **messages** (system, user, assistant), we use `ChatPromptTemplate`.

---

### 🧠 How It Works

This line:

```python
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
```

does the following:

| Line                                                    | Action                                                     |
| ------------------------------------------------------- | ---------------------------------------------------------- |
| `from langchain_core.prompts import ChatPromptTemplate` | Imports the class from the new LangChain structure         |
| `ChatPromptTemplate.from_template(...)`                 | Creates a chat-style prompt from a single user message     |
| `"Tell me a joke about {topic}"`                        | Defines the **template string**, with a variable `{topic}` |

Later, when you run:

```python
response = chain.invoke({"topic": "AI and bananas"})
```

LangChain replaces `{topic}` with `"AI and bananas"` and sends this to the LLM as a chat message from the "user".



###**ChatPromptTemplate**

In [10]:
# 🧠 1. Simple Template with One Variable
prompt = ChatPromptTemplate.from_template("Explain the concept of {topic} in simple terms.")
prompt.invoke({"topic": "black holes"})

ChatPromptValue(messages=[HumanMessage(content='Explain the concept of black holes in simple terms.', additional_kwargs={}, response_metadata={})])

In [11]:
# 💼 2. Professional Email Generator
prompt = ChatPromptTemplate.from_template(
    "Write a professional email to {recipient_name} about {topic}, and make it sound polite and concise."
)
prompt.invoke({
    "recipient_name": "Dr. Martinez",
    "topic": "delayed project timeline"
})

ChatPromptValue(messages=[HumanMessage(content='Write a professional email to Dr. Martinez about delayed project timeline, and make it sound polite and concise.', additional_kwargs={}, response_metadata={})])

In [12]:
# ✍️ 3. Story Starter Prompt
prompt = ChatPromptTemplate.from_template(
    "Write a short story about a {character_type} who discovers a {mystical_object} in the {setting}."
)
prompt.invoke({
    "character_type": "young inventor",
    "mystical_object": "talking crystal",
    "setting": "abandoned laboratory"
})


ChatPromptValue(messages=[HumanMessage(content='Write a short story about a young inventor who discovers a talking crystal in the abandoned laboratory.', additional_kwargs={}, response_metadata={})])

In [13]:
# 🧑‍💼 4. Customer Support Bot
prompt = ChatPromptTemplate.from_template(
    "You are a helpful customer support agent. A customer says: '{customer_message}'. How would you respond?"
)
prompt.invoke({
    "customer_message": "I was charged twice for my subscription."
})


ChatPromptValue(messages=[HumanMessage(content="You are a helpful customer support agent. A customer says: 'I was charged twice for my subscription.'. How would you respond?", additional_kwargs={}, response_metadata={})])

In [14]:
# 🗂 5. System + User Messages
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a wise science tutor."),
    ("human", "Can you explain {concept} like I'm five?")
])

prompt.invoke({"concept": "photosynthesis"})


ChatPromptValue(messages=[SystemMessage(content='You are a wise science tutor.', additional_kwargs={}, response_metadata={}), HumanMessage(content="Can you explain photosynthesis like I'm five?", additional_kwargs={}, response_metadata={})])


## 🧩 What’s Going On in This Example?

```python
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])
```

### 🔍 Breakdown:

You’re creating a **multi-message prompt**, with **different roles**:

* `system`: sets the behavior of the AI
* `human`: simulates user input
* `ai`: simulates the AI's response
* `human`: gives another user message

So you're essentially **building up a conversation** that already has history before the real user input comes in.

---

### 🧠 Why Use This?

This is useful when:

* You want to **set context** for the model (e.g., name, tone)
* You want the model to **see a conversation history** for better coherence
* You’re mimicking a real dialogue in a chatbot

---

### 🎯 The `{}` Braces Are Placeholders

```python
("system", "You are a helpful AI bot. Your name is {name}."),
...
("human", "{user_input}")
```

These get filled in when you call `.invoke(...)`, like so:

```python
prompt_value = template.invoke({
    "name": "Bob",
    "user_input": "What is your name?"
})
```

So the result is a structured message list like:

```python
[
  SystemMessage("You are a helpful AI bot. Your name is Bob."),
  HumanMessage("Hello, how are you doing?"),
  AIMessage("I'm doing well, thanks!"),
  HumanMessage("What is your name?")
]
```

---

### 💡 Key Concept: `ChatPromptValue`

The `.invoke()` method returns a `ChatPromptValue`, which is just a structured object holding the **message sequence**. This is what gets passed into the `ChatOpenAI` model.

---

### 🧠 What You Should Learn From This

* You can build **multi-turn conversations** as input to LLMs.
* `ChatPromptTemplate` accepts a **list of message tuples** like `("role", "message")`.
* You can still use **variables** (`{}` placeholders) in any message.
* This approach gives the model **memory of prior messages** without needing a full memory object (more on that later).


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])


## 🤖 Does `ChatPromptTemplate` Store the Entire Conversation?

### 🔴 Short Answer: **No, it does not *store* the conversation history** — it just *constructs* it at that moment.

What you’re doing in this example:

```python
template = ChatPromptTemplate([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])
```

...is **manually assembling a conversation history** to simulate previous exchanges. This is **static and hardcoded**, not dynamic.

---

### 🧠 So When Does LangChain Actually *Store* Conversation History?

To maintain a *real* running conversation over time (like a chatbot), LangChain provides **memory objects**.

#### ✅ Example: `ConversationBufferMemory`

```python
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)
```

This stores messages dynamically as the chat continues.

You then use it in a chain like:

```python
from langchain.chains import ConversationChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

# Start chatting
response = conversation.predict(input="Hi, who won the World Cup in 2018?")
```

Now, LangChain automatically appends your inputs/outputs to the memory and uses them in future context windows.

---

### ✅ Summary: Manual vs. Automatic History

| Method                     | Stores Conversation? | How?                      |
| -------------------------- | -------------------- | ------------------------- |
| `ChatPromptTemplate`       | ❌ No                 | Static construction       |
| `ConversationBufferMemory` | ✅ Yes                | Automatic message storage |
| `ChatOpenAI` alone         | ❌ No                 | Stateless unless combined |



`ChatPromptTemplate` is **one of the most fundamental building blocks** in LangChain. It's worth getting solid on it before we move on. Here's a final quick-hit summary and a few **advanced tips** so you’re fully equipped:

---

## 🧠 What You Should Know Before Moving On

### ✅ 1. **It Structures Input for Chat Models**

* It builds the list of messages (`system`, `user`, `assistant`) that OpenAI (or similar) models expect
* Supports **placeholders** for dynamic content (`{topic}`, `{user_input}`, `{conversation}`)

---

### ✅ 2. **It’s Reusable and Parameterized**

You define the structure **once**, then `invoke()` it many times with different values. Great for apps, APIs, or workflows.

---

### ✅ 3. **Supports Message Roles and Placeholders**

You can build templates with:

```python
[
  ("system", "You are..."),
  ("human", "{user_input}"),
  ("ai", "{response}"),
  ("placeholder", "{conversation}")
]
```

You can also use:

```python
from langchain_core.prompts import MessagesPlaceholder
```

...for more explicit control.

---

### ✅ 4. **Returns a `ChatPromptValue`**

Which holds the list of structured messages — ready to be passed to an LLM like `ChatOpenAI`.

---

## 🔧 Advanced Tips (You’ll Encounter These Soon)

### 🔄 Supports Jinja2 Templates

You can use more advanced formatting:

```python
ChatPromptTemplate.from_template(
    "Hello {{ name }}! You have {{ messages|length }} new messages."
)
```

This uses [Jinja2](https://jinja.palletsprojects.com/) under the hood if you enable it.

---

### 🧩 Works Seamlessly With:

* `ChatOpenAI` or other chat model classes
* `Memory` (like `ConversationBufferMemory`)
* `Retrieval-Augmented Generation` (e.g., injecting context from search)

---

### 🧱 Common Pattern You’ll See:

```python
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])
```

This lets you:

* Set a role
* Maintain memory (`chat_history`)
* Handle the newest input

This structure powers **most chatbots** built in LangChain.

---

## 🧾 TL;DR: What to Remember

| Thing                | Why It Matters                                          |
| -------------------- | ------------------------------------------------------- |
| `ChatPromptTemplate` | The blueprint for structured prompts                    |
| Placeholders         | Add dynamic input (e.g., `{topic}` or `{conversation}`) |
| `invoke()`           | Fills the template with values                          |
| Output               | A list of chat messages sent to LLMs                    |




Understanding the difference between `ChatPromptTemplate` and `ChatMessagePromptTemplate` helps you write more **modular, flexible prompt code**.


## 🧱 `ChatPromptTemplate`

* **High-level container** for an entire chat prompt.
* Represents a **sequence of messages** (system, human, AI, etc.)
* Used to **combine** multiple `ChatMessagePromptTemplate`s or message strings with roles.

Think of this as the **whole chat history** structure you send to the LLM.

---

### 🔹 `ChatMessagePromptTemplate`

* **Low-level building block** representing **one message** in a chat.
* Used when you want **fine-grained control** over each message.
* You define the role and the template for that one message.

Think of this as **a single line** in the conversation.

---

## 🔁 Analogy

Imagine you're writing a play:

* **`ChatMessagePromptTemplate`** is a single **line of dialogue** (with a speaker).
* **`ChatPromptTemplate`** is the **entire script** — a sequence of those lines.

---

## 🧪 Example to Show the Difference

### 🧩 Using `ChatPromptTemplate` with Plain Tuples (shorthand):

```python
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "What is the weather in {city}?")
])
```

### 🧩 Same Thing, But Using `ChatMessagePromptTemplate`:

```python
from langchain_core.prompts import (
    ChatPromptTemplate,
    ChatMessagePromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

system_msg = SystemMessagePromptTemplate.from_template("You are a helpful assistant.")
human_msg = HumanMessagePromptTemplate.from_template("What is the weather in {city}?")

prompt = ChatPromptTemplate.from_messages([system_msg, human_msg])
```

Result is the same — but this approach lets you **construct or manipulate each message** independently.

---

## 🧠 When to Use Each

| Situation                          | Use `ChatPromptTemplate` | Use `ChatMessagePromptTemplate` |
| ---------------------------------- | ------------------------ | ------------------------------- |
| Quick prototype                    | ✅                        | 🚫                              |
| Full chatbot                       | ✅                        | ✅ (to build message blocks)     |
| Reuse specific message parts       | 🚫                       | ✅                               |
| Need message-level logic           | 🚫                       | ✅                               |
| You like cleaner high-level syntax | ✅                        | 🚫                              |

---

## ✅ Summary

| Object                      | What It Represents                      | Used For                     |
| --------------------------- | --------------------------------------- | ---------------------------- |
| `ChatPromptTemplate`        | The full chat prompt (all messages)     | Final LLM input              |
| `ChatMessagePromptTemplate` | One message template (system, user, ai) | Building reusable components |



In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# Set up the LLM (you'll be prompted for your API key if not set)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# Build a prompt template
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")

# Chain it together
chain = prompt | llm | StrOutputParser()

# Run it
response = chain.invoke({"topic": "AI and bananas"})
print(response)

Why did the AI go bananas? Because it couldn't find its peelings!




## 🧠 What Is `StrOutputParser`?

### 🔹 `StrOutputParser` is a simple output parser

It takes the LLM's **raw output** (usually in a structured format like a `ChatMessage`) and **returns just the text**.

Think of it as:

> “Give me just the plain string response from the model — no wrappers, no formatting, no role metadata.”

---

## 📦 Why Do You Need It?

LangChain breaks the LLM flow into modular parts:

1. **Prompt** → formats the input
2. **LLM** → generates the response (usually a `ChatMessage`)
3. **OutputParser** → extracts usable content (e.g., string, JSON, dict)

Without a parser, you'd get back something like:

```python
AIMessage(content="Sure, here's a joke...")
```

But with `StrOutputParser`, you get:

```python
"Sure, here's a joke..."
```

This makes the result **clean** and easy to print, log, or send to a UI.

---

## ✅ Example in Context

Here’s how it fits into your pipeline:

```python
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
parser = StrOutputParser()

# Chain them together using `|`
chain = prompt | llm | parser

response = chain.invoke({"topic": "AI"})
print(response)
```

Without `StrOutputParser`, the result would be a `ChatMessage` object. With it, it’s just a **clean string**.

---

## 🧱 Summary

| Concept           | Description                                        |
| ----------------- | -------------------------------------------------- |
| `StrOutputParser` | Extracts the `content` field from an LLM response  |
| Input             | `ChatMessage` or model response                    |
| Output            | Plain text (`str`)                                 |
| Use Case          | Clean up the LLM output for display or further use |

---

Later on, you’ll use **other output parsers** too:

* `PydanticOutputParser` → when you want structured JSON objects
* `RetryWithErrorOutputParser` → for validating outputs
* Custom parsers for tools, agents, etc.




Yes, `from langchain_openai import ChatOpenAI` is the LangChain **wrapper for OpenAI's chat models**, like `gpt-3.5-turbo` or `gpt-4`. But LangChain is **model-agnostic**, meaning you can plug in many different LLM providers the same way.

---

## 🧠 What `ChatOpenAI` Does

This class:

* Talks to OpenAI’s API
* Sends structured prompts (`system`, `user`, `assistant`)
* Returns the model’s response

LangChain just wraps it so it plays nicely with the rest of the chain (prompts, memory, tools, etc.).

---

## 🌍 Other Model Wrappers in LangChain

Yes — there are equivalents for most major LLMs. Here’s how it breaks down:

| Model Provider         | LangChain Class                                         | Module to Import                                           |
| ---------------------- | ------------------------------------------------------- | ---------------------------------------------------------- |
| **OpenAI**             | `ChatOpenAI`                                            | `langchain_openai`                                         |
| **Anthropic (Claude)** | `ChatAnthropic`                                         | `langchain_anthropic`                                      |
| **Google (Gemini)**    | `ChatVertexAI` or `ChatGoogleGenerativeAI`              | `langchain_google_vertexai` or `langchain_google_genai`    |
| **Mistral**            | `ChatMistralAI` or `ChatMistral`                        | `langchain_mistralai` or `langchain_community.chat_models` |
| **Llama (Meta)**       | Usually used via `llama.cpp`, Hugging Face, or `ollama` | `langchain_community.chat_models.ChatOllama`               |
| **Cohere**             | `ChatCohere`                                            | `langchain_cohere`                                         |
| **Together.AI**        | `ChatTogether`                                          | `langchain_community.chat_models`                          |
| **Fireworks**          | `ChatFireworks`                                         | `langchain_fireworks`                                      |

These all follow the **same interface** as `ChatOpenAI`, so you can swap them out easily with minor config changes.

---

## 🔁 Swap OpenAI for Another Model

### Example: Using Anthropic’s Claude

```python
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-opus")
```

### Example: Using Local Llama via Ollama

```python
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="llama3")
```

### Example: Using Google Gemini Pro

```python
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro")
```

---

## 🧱 Takeaway

| Component          | Purpose                                                       |
| ------------------ | ------------------------------------------------------------- |
| `ChatOpenAI`       | LangChain's interface to OpenAI’s chat models                 |
| Other chat classes | Provide a unified API to other providers                      |
| Why it matters     | You can **swap providers** without rewriting your entire app! |





## 🧠 What Is the Pipe (`|`) Operator in LangChain?

In LangChain, the pipe operator lets you **chain together components** like:

```python
prompt | llm | parser
```

This creates a **Runnable** — a composable, functional pipeline that passes data from one step to the next.

---

## 🔗 Why Use a Pipe?

It’s:

* ✅ **Readable**: mirrors natural data flow
* ✅ **Modular**: lets you swap in/out components easily
* ✅ **Scalable**: supports arbitrarily complex chains (with memory, agents, tools, etc.)

LangChain’s design philosophy is:

> *Treat every component as a function: `input → output`.*

The pipe combines these "functions" into a **single Runnable chain**.

---

## ⚙️ Under the Hood: Functional Programming

LangChain components like `ChatPromptTemplate`, `ChatOpenAI`, and `StrOutputParser` all implement a common interface: `Runnable`.

So when you do:

```python
chain = prompt | llm | parser
```

It’s equivalent to doing this manually:

```python
formatted_prompt = prompt.invoke({"topic": "AI"})
response = llm.invoke(formatted_prompt)
final_output = parser.invoke(response)
```

The pipe just **composes** those function calls for you.

---

## 📦 What Can Be Piped?

Any object that implements the `Runnable` interface, including:

| Component Type   | Example                        |
| ---------------- | ------------------------------ |
| Prompt templates | `ChatPromptTemplate`           |
| Language models  | `ChatOpenAI`, `ChatAnthropic`  |
| Parsers          | `StrOutputParser`, custom ones |
| Memory           | `ConversationBufferMemory`     |
| Chains           | Custom `RunnableSequence`s     |
| Tools/agents     | Toolchains, ReAct agents       |

---

## 🔁 You Can Pipe More Than Three Things

It’s not limited to 3 components. This is valid too:

```python
template | retriever | summarizer | llm | parser
```

Each one takes the output from the previous and passes its result to the next.

---

## 🧪 Example: Custom Pipe with Side Effects

Let’s say you want to log the prompt before calling the model. You could add a custom `Runnable` in between:

```python
from langchain_core.runnables import RunnableLambda

log_step = RunnableLambda(lambda x: (print("Prompt:", x), x)[1])  # Logs and passes input

chain = prompt | log_step | llm | parser
```

---

## ✅ Summary

| Feature                       | Benefit                           |                                       |
| ----------------------------- | --------------------------------- | ------------------------------------- |
| \`                            | \` operator                       | Connects Runnable components together |
| Acts like a function pipeline | `x → f(x) → g(f(x)) → h(g(f(x)))` |                                       |
| Clean syntax                  | Easy to read and extend           |                                       |
| Core to LangChain             | You’ll use it **everywhere**      |                                       |



You’re ready to explore one of the most exciting parts of LangChain: **agents**. Agents make your LLMs not just talk, but **think**, **decide**, and **act** — like calling tools, doing searches, calculations, or fetching external data.

Let’s first look at some **advanced pipe-based pipelines**, and then see how they look when you plug in an **agent**.

---

## 🔗 Advanced Pipe Example (No Agent Yet)

Let’s say you want an AI that:

1. Takes in a user question
2. Searches documents (via retriever)
3. Passes relevant info into a prompt
4. Gets a model response
5. Parses the output

```python
chain = (
    retriever
    | (lambda docs: {"docs": docs})                     # Convert output to expected input shape
    | prompt_template                                    # Uses "docs" + "question"
    | llm
    | StrOutputParser()
)

response = chain.invoke({"question": "How does photosynthesis work?"})
```



Yes — **exactly!** 🎯 That’s a textbook **RAG** (Retrieval-Augmented Generation) setup — just written using LangChain's elegant pipe-style syntax.

Let’s break it down clearly so you understand *why* this is a RAG and how each part maps to the components of the architecture.

---

## 🔁 What is RAG?

**Retrieval-Augmented Generation** (RAG) is a powerful technique where you:

> 🧠 **Retrieve relevant documents** based on a user’s query, then
> ✍️ **Generate an answer** using both the query and those documents.

This bridges the gap between what a model knows **(pretraining)** and what it needs to answer **(current or external info)**.

---

## ✅ LangChain RAG Breakdown

Here’s your pipeline:

```python
chain = (
    retriever
    | (lambda docs: {"docs": docs})                     
    | prompt_template                                    
    | llm
    | StrOutputParser()
)
```

### 🔹 `retriever`

* Accepts the user's **query**
* Returns a list of **relevant documents**
* These docs are usually from a vector store (e.g., Chroma, FAISS)

### 🔹 `(lambda docs: {"docs": docs})`

* A quick shim to **rename the output** into the expected input format for the next step (e.g., if your prompt expects `docs` as a key)
* Think of this like: `retrieved_docs → {"docs": retrieved_docs}`

### 🔹 `prompt_template`

* A `ChatPromptTemplate` that expects inputs like `{"docs": ..., "question": ...}`
* This creates a **final structured prompt** like:

  ```
  You are a helpful assistant. Use the following documents:

  {doc content here}

  Question: {question}
  ```

### 🔹 `llm`

* Your model (e.g., `ChatOpenAI`) that generates the response

### 🔹 `StrOutputParser()`

* Extracts just the **text content** of the model's response

---

## 📦 Input and Output

### Input:

```python
{"question": "How does photosynthesis work?"}
```

### Output:

```text
"Photosynthesis is the process by which plants use sunlight to..."
```

Behind the scenes, the model is generating this answer **using the retrieved documents**, not just its pretrained knowledge.

---

## ✅ Why This Is RAG

| RAG Element           | In Your Pipeline           |
| --------------------- | -------------------------- |
| **Retriever**         | `retriever`                |
| **Prompt-building**   | `lambda + prompt_template` |
| **Generator**         | `llm`                      |
| **Result formatting** | `StrOutputParser()`        |

✔️ You’re using **external knowledge** to ground your LLM
✔️ You’re controlling the **prompt input**
✔️ You’re retrieving **on the fly** at runtime




